# Prediction <img align="right" src="../../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">

## Description
1. Generate predictions using the imported model at a number of test locations and save results to disk

### Load Packages

In [ ]:
import sys
import datacube
import numpy as np
import xarray as xr
import geopandas as gpd
import subprocess as sp
from joblib import load
import matplotlib.pyplot as plt
from datacube.utils import geometry
from datacube.utils.cog import write_cog
from datacube.utils.geometry import assign_crs
from sklearn.preprocessing import StandardScaler
from odc.algo import xr_geomedian

sys.path.append('../../Scripts')
from deafrica_datahandling import load_ard
from deafrica_bandindices import calculate_indices
from deafrica_dask import create_local_dask_cluster
from deafrica_classificationtools import HiddenPrints
from deafrica_plotting import rgb, display_map, map_shapefile
from deafrica_classificationtools import HiddenPrints, predict_xr
from deafrica_spatialtools import xr_rasterize

from feature_layer_functions import gm_mads_two_seasons
from datacube.testutils.io import rio_slurp_xarray
import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

### Set up a dask cluster
This will help keep our memory use down and conduct the analysis in parallel. If you'd like to view the dask dashboard, click on the hyperlink that prints below the cell. You can use the dashboard to monitor the progress of calculations.

In [ ]:
create_local_dask_cluster()

## Analysis parameters

* `model_path`: The path to the location where the model exported from the previous notebook is stored
* `training_data`: Name and location of the training data `.txt` file output from runnning `1_Extract_training_data.ipynb`
* `features_scaled`: 
* `sc_path`: Use this parameter to indicate whether or not the features where scaled using the `standardScalar()` method in the previous notebook. If you did scale the features then provide the path to the `standardScalar` values output in the previous notebook. Otherwise, set this parameters to `None`
* `test_shapefile`: A shapefile containing polygons that represent regions where you want to test your model. The shapefile should have a unique identifier as this will be used to export classification results to disk as geotiffs.
* `results`: A folder location to store the classified geotiffs 

In [ ]:
model_path = 'results/gm_mads_two_seasons_ml_model_20201123.joblib'

training_data = "results/training_data/gm_mads_two_seasons_training_data_20201123.txt"

sc_path = None #'results/std_scaler.bin'

test_shapefile = 'data/eastern_testing_sites.geojson'

results = 'results/classifications/'

model_type='gm_mads_two_seasons'

### Connect to the datacube

In [ ]:
dc = datacube.Datacube(app='prediction')

### Open and inspect test_shapefile

In [ ]:
gdf = gpd.read_file(test_shapefile)

# gdf.head()
map_shapefile(gdf, attribute='GRID_ID')

## Open the model

If we ran the optional feature scaling method in the `3_Train_fit_evaluate_classifier.ipynb`, then we will also load in the standard scalar values.

The code below will also re-open the training data we exported from `2_Inspect_training_data.ipynb` and grab the column names (features we selected).

In [ ]:
model = load(model_path)

In [ ]:
# load the column_names
with open(training_data, 'r') as file:
    header = file.readline()
    
column_names = header.split()[2:]
print(column_names)

## Making a prediction


### Set up datacube query

These query options should match the query params in `1_Extract_training_data.ipynb`, unless there are measurements that no longer need to be loaded because they were dropped during the feature selection process.

In [ ]:
#set up our inputs to collect_training_data
products =  ['s2_l2a']
time = ('2019-01','2019-12')

# Set up the inputs for the ODC query
measurements =  ['red','blue','green','nir','swir_1','swir_2','red_edge_1','red_edge_2', 'red_edge_3']
resolution = (-20,20)
output_crs='epsg:6933'
dask_chunks={}

### Loop through test tiles and predict

For every tile we list in the `test_shapefile`, we calculate the feature layers, and then use the DE Africa function `predict_xr` to classify the data.

The results are exported to file as Cloud-Optimised Geotiffs.

In [ ]:
query = {
    'time': time,
    'measurements': measurements,
    'resolution': resolution,
    'output_crs': output_crs,
    'group_by' : 'solar_day',
}

In [ ]:
%%time
for index, row in gdf.iterrows():
    
    #get id for labelling
    g_id=gdf.iloc[index]['GRID_ID']
    print('working on grid: ' + g_id)
    
    # Get the geometry
    geom = geometry.Geometry(geom=gdf.iloc[index].geometry,
                             crs=gdf.crs)

     # generate a datacube query object
    query = {
        'time': time,
        'measurements': measurements,
        'resolution': resolution,
        'output_crs': output_crs,
        'group_by' : 'solar_day',
    }
    
    # Update dc query with geometry      
    query.update({'geopolygon': geom}) 
    
    #load data
    with HiddenPrints():
        ds = load_ard(dc=dc,
                      products=products,
                      dask_chunks=dask_chunks,
                      **query)

    #calculate features
    data = gm_mads_two_seasons(ds)
    #predict using the imported model
    predicted = predict_xr(model,
                           data,
                           proba=True,
                           persist=True,
                           clean=True,
                           return_input=True
                          ).compute()
    
    # Mask dataset to set pixels outside the polygon to `NaN`
#     with HiddenPrints():
#         mask = xr_rasterize(gdf.iloc[[index]], ds)
#         predicted = predicted.where(mask)
    
    #grab just the predictions for post process filtering
    predict = predicted.Predictions
    
    #mask with WOFS
    wofs_query=query.pop('measurements')
    wofs=dc.load(product='ga_ls8c_wofs_2_summary',**query)
    wofs = wofs.frequency > 0.2 # threshold
    predict=predict.where(~wofs, 0)    
    
    #mask steep slopes
    url_slope = "https://deafrica-data.s3.amazonaws.com/ancillary/dem-derivatives/cog_slope_africa.tif"
    slope = rio_slurp_xarray(url_slope, gbox=ds.geobox)
    slope = slope > 35
    predict=predict.where(~slope, 0)
    predict=predict.astype(np.int16)
    
    #export classifications to disk
    write_cog(predict, results+'predicted/Eastern_tile_'+g_id+'_prediction_pixel_'+model_type+'_20201123.tif', overwrite=True)
    write_cog(predicted.Probabilities, results+ 'proba/Eastern_tile_'+g_id+'_probabilities_pixel_'+model_type+'_20201123.tif', overwrite=True)
    
    #remove predict and proba so we can export the inputs
    del predicted['Probabilities']
    del predicted['Predictions']
    
    #export inputs
    predicted.to_netcdf(results+ 'input/Eastern_tile_'+g_id+'_inputs.nc')
    

In [ ]:
# predict.plot(size=15)

In [ ]:
import xarray as xr
x=xr.open_dataset(results+ 'input/Eastern_tile_'+g_id+'_inputs.nc').squeeze()
x#.plot(figsize=(12, 12));